# 1.Import Dependencies

In [ ]:
import os
import gymnasium
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [ ]:
environment_name = 'CartPole-v1'
env = gymnasium.make(environment_name, render_mode="human")

In [ ]:
# episodes = 5
# for episode in range(1, episodes + 1):
#     state = env.reset()
#     done = False
#     score = 0
# 
#     while not done:
#         action = env.action_space.sample()
#         n_state, reward, terminated, truncated, info = env.step(action)
#         done = terminated or truncated
#         score += reward
# 
#     print(f'Episode:{episode}, Score:{score}')
# env.close()

# 3.Understanding the Environment

In [ ]:
env.action_space

In [ ]:
env.action_space.sample()

In [ ]:
env.observation_space.sample()

# 4. Train Model

In [ ]:
log_path = os.path.join('Training', 'Logs')


In [ ]:
import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    print("MPS is not available, using CPU.")



In [ ]:
env = gymnasium.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, device=device)

In [ ]:
model.learn(total_timesteps=20000)

# 5. Save Model & Load Model

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_CartPole')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env=env)

# 6. Evaluate

In [ ]:
# Step 1: Recreate the environment
env = gymnasium.make("CartPole-v1", render_mode="human")

# Step 2: Define the path to the saved model
PPO_path = os.path.join("Training", "Saved Models", "PPO_model_CartPole")

# Step 3: Load the saved model
model = PPO.load(PPO_path, env=env)

# Step 4: Custom evaluation function
def evaluate_policy_with_rendering(model, env, n_eval_episodes=10, render=True):
    """
    Evaluate the policy of a loaded model with optional rendering.
    """
    episode_rewards = []

    for episode in range(n_eval_episodes):
        reset_output = env.reset()
        state = reset_output[0] if isinstance(reset_output, tuple) else reset_output
        done = False
        total_reward = 0

        while not done:
            if render:
                env.render()  # Render each frame

            # Get the action from the model
            action, _states = model.predict(state, deterministic=True)
            
            # Adjust for environments returning 4 or 5 values
            step_output = env.step(action)
            if len(step_output) == 5:
                state, reward, terminated, truncated, info = step_output
            else:
                state, reward, terminated, truncated = step_output

            # Convert reward to scalar to avoid warnings
            total_reward += reward.item() if hasattr(reward, 'item') else float(reward)

            # Combine termination flags
            done = terminated or truncated

        episode_rewards.append(total_reward)
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    # Calculate mean and standard deviation of rewards
    mean_reward = sum(episode_rewards) / n_eval_episodes
    std_reward = (sum([(x - mean_reward) ** 2 for x in episode_rewards]) / n_eval_episodes) ** 0.5

    return mean_reward, std_reward

# Step 5: Evaluate the loaded model
mean_reward, std_reward = evaluate_policy_with_rendering(model, env, n_eval_episodes=10, render=True)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# Close the environment
env.close()

# 7. Test Model

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO

# Create the environment
env = gym.make("CartPole-v1", render_mode="human")

# Define the path to the saved model
PPO_path = os.path.join("Training", "Saved Models", "PPO_model_CartPole")

# Load the saved model
model = PPO.load(PPO_path, env=env)

# Number of episodes
episodes = 10

for episode in range(1, episodes + 1):
    # Extract the observation from the reset output
    obs, _ = env.reset()
    done = False
    score = 0

    while not done:
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward

    print(f'Episode: {episode}, Score: {score}')

# Close the environment
env.close()

# 8. View Logs in Tensorboard

In [5]:
import os
log_path = os.path.join('Training', 'Logs')
training_log_path = os.path.join(log_path, 'PPO_2')


In [6]:
!tensorboard --logdir={training_log_path}

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
W1220 12:48:48.656814 6127382528 application.py:559] path /apple-touch-icon-precomposed.png not found, sending 404
W1220 12:48:48.712542 6127382528 application.py:559] path /apple-touch-icon.png not found, sending 404
^C
